<a href="https://colab.research.google.com/github/kmt19/siim_otani/blob/master/drive2gcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.9MB/s 


In [0]:
#https://drive.google.com/drive/u/2/my-drive/
dir_id = "1zHVf-O0xXgd47GaVjkpFRUuq59-4Ci5h"

# フォルダ内にあるファイル名とファイルIDを表示する
file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % dir_id})
for f in file_list :
  new_list=drive.ListFile({'q': "'%s' in parents and trashed=false" % f[0]["id"]}).GetList()
  a=drive.CreateFile({'id': new_list[0]["id"],'q': "'%s' in parents and trashed=false" % new_list[0]["id"]})
  print(a)
  a.GetContentFile('sample.dcm')
  break
!ls

GoogleDriveFile({'id': '1KAnyCwq8q8shjI_3wO2fN2P5s07kbssh', 'q': "'1KAnyCwq8q8shjI_3wO2fN2P5s07kbssh' in parents and trashed=false"})


FileNotDownloadableError: ignored

In [0]:
from google import colab
colab.drive.mount('/content/gdrive')
!ls 'gdrive/My Drive'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'Colab Notebooks'    dicom-images-train
 dicom-images-test   手法のアイデア.gdoc


In [0]:
!pip install pydicom
import os
import pydicom
from glob import glob
from tqdm import tqdm
path = 'gdrive/My Drive/dicom-images-train'
folders = os.listdir(path)
for folder in folders:
  new_folder = path + '/' + folder
  new_folder = new_folder + '/' + os.listdir(new_folder)[0]
  new_file = new_folder + '/' + os.listdir(new_folder)[0]
  ds = pydicom.dcmread(new_file)
  break